In [1]:
import numpy
import math
from sortedcontainers import SortedDict
import random
import numpy 
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import truncnorm


In [2]:
# Code for generating positions, distances, travel times and preferences
def generate_positions(males, x_dim, y_dim):
    Xs = numpy.random.rand(males) * x_dim
    Ys = numpy.random.rand(males) * y_dim
    return [Xs,Ys]

def compute_distances_travel_times(males, positions, bird_speed):
    male_dist = numpy.zeros((males, males))
    travel_times = numpy.zeros((males, males))
    for i in range(males):
        for j in range(i + 1, males):
            dist = math.sqrt((positions[0][j] - positions[0][i]) ** 2 + (positions[1][j] - positions[1][i]) ** 2)
            travel = dist / bird_speed
            male_dist[j][i] = dist
            male_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return (male_dist, travel_times)

def compute_visit_preferences(males, distances, lambda_dist):
    # compute exponential of each coefficient
    visit_preferences = numpy.exp(-lambda_dist * distances)
    # remove the identity matrix (exp(0) = 1)
    visit_preferences = visit_preferences - numpy.eye(males)
    # make rows sum to one
    visit_preferences = (visit_preferences.transpose() / numpy.sum(visit_preferences, 1)).transpose()
    return visit_preferences

# functions to generate tickets and manage timeline
def generate_ticket(start_time, end_time, length_activity, owner, action, target):
    global timeline
    ticket = {"start_time": start_time,
              "end_time": end_time,
              "length_activity": length_activity,
              "owner": owner,
              "action": action,
              "target": target
             }
    # now add to timeline
    timeline[(ticket["end_time"], ticket["owner"])] = ticket

def draw_foraging_time(start_time):
    time_between = FG_tau_std * truncnorm.rvs(FG_tau_norm_range[0], FG_tau_norm_range[1]) + FG_tau_mean
    return start_time + time_between

def action_forage(bird_id, current_time):
    global birds
    # generate the time it takes to forage
    time_spent_foraging = numpy.random.gamma(FG_k, FG_theta)/FG_divisor
    time_action_ends = current_time + time_spent_foraging
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_foraging,
                   owner = bird_id,
                   action = "foraging",
                   target = -1)
    # update the bird:
    my_bird = birds[bird_id]
    my_bird["current_state"] = "foraging"
    my_bird["action_starts"] = current_time
    my_bird["action_ends"] = time_action_ends
    my_bird["foraging_time_data"][0] += 1 # add one more foraging event
    my_bird["foraging_time_data"][1] += time_spent_foraging # add the time spent foraging
    my_bird["foraging_time_data"][2] += time_spent_foraging ** 2 #add the time spent foraging squared
    # update the time to next foraging: start counting when foraging ended
    birds[bird_id]["next_foraging_time"] = draw_foraging_time(time_action_ends)

def action_stay_at_bower(bird_id, current_time):
    global birds
    # generate the length of the stay
    time_spent_at_bower = RBSB_tau_std * truncnorm.rvs(RBSB_tau_norm_range[0], RBSB_tau_norm_range[1]) + RBSB_tau_mean
    time_action_ends = current_time + time_spent_at_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_at_bower,
                   owner = bird_id,
                   action = "staying at bower",
                   target = -1)
    # update the bird:
    my_bird = birds[bird_id]
    my_bird["current_state"] = "staying at bower"
    my_bird["action_starts"] = current_time
    my_bird["action_ends"] = time_action_ends
    my_bird["staying_time_data"][0] += 1 # add one more staying at bower event
    my_bird["staying_time_data"][1] += time_spent_at_bower # add the time spent at bower
    my_bird["staying_time_data"][2] += time_spent_at_bower ** 2 #add the time spent at bower squared

def action_travel_to_maraud(bird_id, current_time):
    global birds
    # choose who to maraud
    tmp = numpy.random.rand()
    target = numpy.argwhere(birds[bird_id]["travel_preferences"] > tmp)[0][0] 
    time_to_travel = birds[bird_id]["travel_times"][target]
    time_action_ends = current_time + time_to_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_to_travel,
                   owner = bird_id,
                   action = "travel to maraud",
                   target = target)
    # update the bird:
    my_bird = birds[bird_id]
    my_bird["current_state"] = "travel to maraud"
    my_bird["action_starts"] = current_time
    my_bird["action_ends"] = time_action_ends
    my_bird["traveling_time_data"][0] += 1 # add one more travel event
    my_bird["traveling_time_data"][1] += time_to_travel * 2 # add the time spent traveling
    my_bird["traveling_time_data"][2] += (time_to_travel ** 2) * 2 #add the time spent traveling squared
    #NOTE: use * 2 to account for return time, as well
    
def action_repair_bower(bird_id, current_time):
    global birds
    # generate the length of the repair bout
    time_spent_repairing_bower = RBSB_tau_std * truncnorm.rvs(RBSB_tau_norm_range[0], RBSB_tau_norm_range[1]) + RBSB_tau_mean
    time_action_ends = current_time + time_spent_repairing_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_repairing_bower,
                   owner = bird_id,
                   action = "repairing bower",
                   target = -1)
    # update the bird:
    my_bird = birds[bird_id]
    my_bird["current_state"] = "repairing bower"
    my_bird["action_starts"] = current_time
    my_bird["action_ends"] = time_action_ends
    my_bird["repairing_time_data"][0] += 1 # add one more travel event
    my_bird["repairing_time_data"][1] += time_spent_repairing_bower # add the time spent traveling
    my_bird["repairing_time_data"][2] += time_spent_repairing_bower ** 2 #add the time spent traveling squared
    # note: already accounts for the improvements
    my_bird["bower_state"] +=  time_spent_repairing_bower
    # cannot make it better than 0
    if my_bird["bower_state"] > 0.0:
        my_bird["bower_state"] = 0.0

def action_maraud(marauder_id, marauder_target, current_time):
    global birds
    # note: HARD CODED PARAMS!
    time_spent_marauding = 0.1 
    # note: HARD CODED PARAMS!
    damage_to_bower = 6.0 
    time_action_ends = current_time + time_spent_marauding
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_marauding,
                   owner = marauder_id,
                   action = "marauding",
                   target = marauder_target)
    # update marauder
    my_bird = birds[marauder_id]
    my_bird["current_state"] = "marauding"
    my_bird["action_starts"] = current_time
    my_bird["action_ends"] = time_action_ends
    my_bird["marauding_time_data"][0] += 1 # add one more travel event
    my_bird["marauding_time_data"][1] += time_spent_marauding # add the time spent traveling
    my_bird["marauding_time_data"][2] += time_spent_marauding ** 2 #add the time spent traveling squared
    # update target
    birds[marauder_target]["bower_state"] = birds[marauder_target]["bower_state"] - damage_to_bower
    

def action_travel_from_maraud(marauder_id, marauder_target, current_time):
    global birds
    time_from_travel = birds[marauder_id]["travel_times"][marauder_target]
    time_action_ends = current_time + time_from_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_from_travel,
                   owner = marauder_id,
                   action = "travel from maraud",
                   target = marauder_target)
    # update the bird:
    my_bird = birds[marauder_id]
    my_bird["current_state"] = "travel from maraud"
    my_bird["action_starts"] = current_time
    my_bird["action_ends"] = time_action_ends

    
def action_mating_attempt(female_id, current_time):
    global birds
    global female_birds
    female_index = int(female_id[1:]) #convert from ID to index; i.e. "F0" -> 0
    female = female_birds[female_index] # extract female
    last_location = female["already_visited"][-1] #index of most recently visited male
    p = birds[last_location]["travel_preferences"].copy()
    # this line "undoes" the cumulative sum
    p = numpy.diff(numpy.concatenate((numpy.array([0]), p)))
    extra_wait = 0.0
    if len(female["already_visited"]) == female["max_per_day"]:
        female["already_visited"] = []
        extra_wait = female["wait_period"]  
    tmp = numpy.random.rand()
    p = numpy.cumsum(p)
    scale_rand = p[-1]
    tmp = numpy.random.rand() * scale_rand
    target = numpy.argwhere(p > tmp)[0][0]
    time_to_travel = birds[last_location]["travel_times"][target]
    time_action_ends = current_time + time_to_travel
    generate_ticket(start_time = time_action_ends + extra_wait, #HARD CODE
                    end_time = time_action_ends + extra_wait,
                    length_activity = time_to_travel + extra_wait,
                    owner = female_id,
                    action = "mating attempt",
                    target = target)

def initialize_male(bird_id, bird_strategy, bird_xy, bird_preferences, bird_travel_times):
    # initialize dictionary
    bird = {"id": bird_id,
            "current_state": "none",
            "action_starts": 0.0,
            "action_ends": -1.0,
            "probability_maraud": bird_strategy,
            "bower_state": 0.0,
            "successful_mating": 0,
            "next_foraging_time": draw_foraging_time(0.0),
            "travel_preferences": numpy.cumsum(bird_preferences), # note: store cumulative probability for faster choice
            "travel_times": bird_travel_times, 
            "position": bird_xy,
            "foraging_time_data": [0.0, 0.0, 0.0], #number of events, cumulative time spent, sum(duration^2)
            "staying_time_data": [0.0, 0.0, 0.0],
            "repairing_time_data": [0.0, 0.0, 0.0],
            "marauding_time_data": [0.0, 0.0, 0.0],
            "traveling_time_data": [0.0, 0.0, 0.0]
            }
    return(bird)

def initialize_female(female_id, males):
    #initialize dictionary
    female_bird = {"id": female_id,
             "already_visited": [numpy.random.randint(males)], # choose a random male to be the "last visited"
             "max_per_day": min(males - 1, 6),
             "wait_period": 12,
            #HARD CODED PARAMS
            }
    return(female_bird)

# this is the most important function!
def read_ticket(tic):
    global birds
    global t_max
    if tic["action"] in ("foraging", "staying at bower", "repairing bower", "travel from maraud"):
        # I am back at the bower, choose new action
        choose_action(birds[tic["owner"]], tic["end_time"])
    elif tic["action"] == "travel to maraud":
        # check whether the target is at home
        go_maraud = True
        if birds[tic["target"]]["current_state"] in ("staying at bower", "repairing bower"):
            go_maraud = False
        if birds[tic["target"]]["bower_state"] < 0.0:
            go_maraud = False
        if go_maraud: # maraud
            action_maraud(tic["owner"], tic["target"], tic["end_time"])
        else: # go back
            action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    elif tic["action"] == "marauding":
        # travel back
        action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    elif tic["action"] == "mating attempt":
        if birds[tic["target"]]["current_state"] == "staying at bower": #if male is at bower and it is intact
            birds[tic["target"]]["successful_mating"] += 1 #successfully mate and stop generating tickets
        else:  
            female_birds[int(tic["owner"][1:])]["already_visited"].append(tic["target"]) #update the female's already_visited list
            if tic['end_time'] < t_max:
                action_mating_attempt(tic["owner"], tic["end_time"]) #generate a new ticket
    else:
        1 / 0 # something went horribly wrong
        
# this function should be called every time the bird 
# 1) is back to the bower (from foraging, marauding), 
# 2) has finished repairing the bower, 
# 3) or has finished a stint at staying at bower
def choose_action(bird, current_time):
    global t_max
    # stop generating actions at t_max
    if current_time < t_max:
        # if it's time to eat
        if current_time > bird["next_foraging_time"]:
            action_forage(bird["id"], current_time)
        # if the bower needs repair
        elif bird["bower_state"] < 0.0:
            # go repair
            action_repair_bower(bird["id"], current_time)
        # check if it wants to maraud
        elif numpy.random.rand() < bird["probability_maraud"]:
            # go maraud
            action_travel_to_maraud(bird["id"], current_time)
        else:
            # stay at bower
            action_stay_at_bower(bird["id"], current_time)

In [3]:
# Global Variables

# TIMELINE
timeline = SortedDict()
t_max = 12*10 #12 * 30 # time when simulation ends

# MALES
males = 20 # number of male birds

# FEMALES
F_per_M = 3 #The number of sexualy mature females per sexually mature male
females = males * F_per_M # number of female birds
female_visit_param = [0, t_max / 2.0] # females visit early in the period

# POSITIONS AND TRAVEL TIME
x_dim, y_dim = 2000, 500 # dimensions of environment
bird_speed = 12 * 3600 # m/hr (12 m/s)
# now choose lambda_dist, controlling the probability of traveling to a neighbor
# the probability of choosing a neighbor at distance x is proportional to exp(-\lambda x)
# choose lambda such that 99% of the mass is before 800 meters
improb = 0.99
improb_distance = 800
lambda_dist = - math.log(1.0 - improb) / improb_distance

# ACTION DISTRIBUTIONS
# Time of forage
FG_tau_mean, FG_tau_std = .4, .167 #mean and sd of truncated normal distribution rv to find a male's time until next FG
FG_tau_range = [0, 1] #maximum and minimum FG taus
FG_tau_norm_range = [(FG_tau_range[0] - FG_tau_mean) / FG_tau_std, (FG_tau_range[1] - FG_tau_mean) / FG_tau_std] #normalized
# Duration of forage
FG_k=1.5 #the shape of the gamma distribution rv used to generate FG taus
FG_theta=5 #the scale of the gamma distribution rv used to generate FG taus
FG_divisor=60
# Duration of repair bower / stay at bower
RBSB_tau_mean, RBSB_tau_std = .1583, .09755 #mean and sd of truncated normal distribution rv to find duration of repair bower / stay at bower
RBSB_tau_range = [0,.5] #maximum and minimum taus
RBSB_tau_norm_range = [(RBSB_tau_range[0] - RBSB_tau_mean) / RBSB_tau_std, (RBSB_tau_range[1] - RBSB_tau_mean) / RBSB_tau_std] #normalized

num_sims=5



strategies = numpy.random.random(males)

In [4]:
random.seed(1)

dist_vec = [500]#, 1000]#, 500, 750, 1000]
prop_vec = numpy.linspace(0.1, 0.9, num = 3)

maraud_fit_dist=[[0.0]*len(prop_vec)]*len(dist_vec)
sd_maraud_fit_dist=[[0.0]*len(prop_vec)]*len(dist_vec)
#guard_fit_dist=[[0.0]*len(prop_vec)]*len(dist_vec)
for x in range(len(dist_vec)):
    print("dist {:f}/{:f}".format(x, len(dist_vec)))
    maraud_fit_prop=[0.0]*len(prop_vec)
    sd_maraud_fit_prop=[0.0]*len(prop_vec)
    #guard_fit_prop=[0.0]*len(prop_vec)
    for y in range(len(prop_vec)):
        print("prop {:f}/{:f}".format(y, len(prop_vec)))
        prop_marauder=prop_vec[y] #proportion of marauders in population #put w parameters later
        marauder_fitness=[[0.0]*males]*num_sims
        for j in range(num_sims): #for each simulation...
            print("sim {:f}".format(j))
            # BIRDS
            birds = []
            female_birds=[]
            strategies=numpy.array([0]*males) #initialize strategy states, 0=guarder, 1 will = marauder
            marauder_inds=numpy.random.choice(list(range(males)), size=int(males*prop_marauder), replace=False) #choose the marauders
            strategies[marauder_inds]=.3 #set these individuals to be marauders 
            #strategies = numpy.random.random(males)
            # initialize positions, travel times and preferences
            x_dim=dist_vec[x]
            y_dim=dist_vec[x]
            positions=generate_positions(males, x_dim, y_dim)
            distances, travel_times=compute_distances_travel_times(males,positions,bird_speed) #these are the male distance and travel time mats
            visit_preferences=compute_visit_preferences(males, distances, lambda_dist)
            #initialize males
            for i in range(males):
                birds.append(initialize_male(i, 
                                             strategies[i], 
                                             (positions[0][i], positions[1][i]), 
                                             visit_preferences[i],
                                             travel_times[i]))
                # choose its first action
                choose_action(birds[-1], 0.0)
            #initialize females
            for i in range(females): #females
                female_id = "F" + str(i)
                female_birds.append(initialize_female(female_id, males)) #female IDs start where males end (if there are 10 males, the first female would be 11)
                #choose time for initial mating attempt
                first_time = numpy.random.uniform(female_visit_param[0], female_visit_param[1])
                action_mating_attempt(female_id, first_time)
            # this is the main loop
            while len(timeline) > 0:
                current_ticket = timeline.popitem(0)
                read_ticket(current_ticket[1])
                # for debug: store all the past tickets
                #past_events.append(current_ticket)
                # and every so often check the length of the timeline
                #if numpy.random.rand() < 0.01:
                    #timeline_lengths.append(len(timeline))
            fitness_states=[]
            for mar in marauder_inds:
                fitness_states.append(birds[mar]["successful_mating"])
            marauder_fitness[j]=numpy.mean(fitness_states)
            #ia = numpy.indices(fitness_states.shape)
            #not_indices = numpy.setxor1d(ia, marauder_inds)
            #guarder_fitness[j]=numpy.mean(avsfs[not_indices])
        maraud_fit_prop[y]=numpy.mean(marauder_fitness)
        sd_maraud_fit_prop[y]=numpy.std(marauder_fitness)
        #guard_fit_prop[y]=numpy.mean(guarder_fitness)
    maraud_fit_dist[x]=maraud_fit_prop
    sd_maraud_fit_dist[x]=sd_maraud_fit_prop
    #guard_fit_dist[x]=guard_fit_prop
    

            





dist 0.000000/2.000000
prop 0.000000/9.000000
sim 0.000000
sim 1.000000
sim 2.000000
sim 3.000000
sim 4.000000
sim 5.000000
sim 6.000000
sim 7.000000
sim 8.000000
sim 9.000000
sim 10.000000
sim 11.000000
sim 12.000000
sim 13.000000
sim 14.000000
sim 15.000000
sim 16.000000
sim 17.000000
sim 18.000000
sim 19.000000
sim 20.000000
sim 21.000000
sim 22.000000
sim 23.000000
sim 24.000000
sim 25.000000
sim 26.000000
sim 27.000000
sim 28.000000
sim 29.000000
sim 30.000000


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig, ax=plt.subplots()
for x in range(len(dist_vec)): 
    series=maraud_fit_dist[x]
    ax.plot(prop_vec, series, linestyle='-', marker='o', label="d={:f}".format(dist_vec[x]))
    #plt.errorbar()
legend=ax.legend(loc=1)
plt.xlabel("Proportion marauders")
plt.ylabel("Average marauder mating success")
plt.axhline(y=3, ls=':', color='black')
plt.show() 
